In [ ]:
import os
import subprocess

# List of PDB files to process
pdb_files = [
    "/home/hp/nayanika/github/GPX6/prep_structures/humansec/D25N.pdb",
    "/home/hp/nayanika/github/GPX6/prep_structures/humansec/G27E.pdb",
    # Add more PDB files as needed
]

# Base directory where you want to create folders
base_scr_dir = "/home/hp/results/humansec"

# Directories for the input files
run_dir = "/home/hp/nayanika/github/GPX6/cluster_scripts"
genrelax_dir = "/home/hp/nayanika/github/GPX6/input"
topology_dir = "/home/hp/nayanika/github/GPX6/prep_structures/humansec"
fep_dir = "/home/hp/nayanika/github/GPX6/input/fep"  # Correct FEP directory path

# Create the mousecys directory if it does not exist
os.makedirs(base_scr_dir, exist_ok=True)

# Loop through each PDB file
for pdb_file in pdb_files:
    # Extract the system name from the PDB file (removing the extension)
    system_name = os.path.basename(pdb_file).replace(".pdb", "")

    # Define the topology file path
    topology_file = os.path.join(topology_dir, f"{system_name}.top")
    
    # Check if the topology file exists
    if not os.path.isfile(topology_file):
        print(f"No topology file found for {system_name} at {topology_file}")
        continue

    # Create a directory for the system in the base output directory
    system_dir = os.path.join(base_scr_dir, system_name)
    os.makedirs(system_dir, exist_ok=True)

    # Remove any existing replicas for a clean start (uncomment if needed)
    # os.system(f"rm -rf {os.path.join(system_dir, 'replica*')}")

    # Check if the .pdb file exists
    if os.path.isfile(pdb_file):
        # Construct the command to run q_genrelax.py
        command = [
            "q_genrelax.py", 
            os.path.join(genrelax_dir, "selgenrelax.proc"),
            "--top", topology_file,
            "--pdb", pdb_file,
            "--fep", os.path.join(fep_dir, "GPX6_wthumansec.fep"),
            "--outdir", os.path.join(system_dir, "minim"),
            "--rs", os.path.join(run_dir, "run_qdyn_5.sh")
        ]
        
        # Run the command
        try:
            subprocess.run(command, check=True)
            print(f"Successfully processed {pdb_file} with topology {topology_file}")
        except subprocess.CalledProcessError as e:
            print(f"Error processing {pdb_file}: {e}")
    else:
        print(f"No .pdb file found at {pdb_file}")